In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import heapq
import cmath
import warnings
from enum import Enum
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from collections import Counter
warnings.filterwarnings("ignore")

In [ ]:
class Compression_Method(Enum):
    XY = 1               #applies PCA on X and Y then filters (1)
    AmpPhase = 2         #applies PCA on Amplitude and Phase then filters (2)
    AmpPhaseFiltered = 3 #applies PCA on Amplitude and Phase after filtering (3)

#Modify this to change the approach used: XY, AmpPhase, AmpPhaseFiltered
method = Compression_Method.AmpPhase
scaler = StandardScaler()
ignorePhases = True
saveCSV = True

base_directory = 'csv/step_' + str(method.value)
os.makedirs(base_directory, exist_ok=True)

#### Variables and Functions definitions

In [ ]:
notInterestedIndexes = list(range(-32,-28)) + list(range(0,1)) + list(range(29,32)) #null columns in the dataset
interestedIndexes = list(range(-28,0)) + list(range(1,29)) #non null columns in the dataset

w1=5 #for filtering
w2=3 #for windows
lambda1=3 #threshold

#for ground truth
t2,t3,t4 = [570.0, 873.0, 1254.0] #separation times for gt
gt1 = [120,180,240,300,390,540]
gt2 = [t2+i for i in range(60,300,30)]
gt3 = [t3+30,t3+57] + [t3+i for i in range(90,390,30)]
gt4 = [t4+i for i in range(30,630,30)]
lower_bounds = gt1+gt2+gt3+gt4
upper_bounds = [l + 1 for l in lower_bounds]

In [ ]:
def checkGT(predicted_true,predicted_false):
    tp,tn,fp,fn = [0,0,0,0]
    
    for t in predicted_true:
        fp = fp+1  #add false positive
        for l in lower_bounds:
            if abs(t-l) <=w2:
                tp = tp+1 #put true positive
                fp = fp-1#remove the old false positive
                break
                
    for t in predicted_false:
        tn= tn+1  #add false positive
        for l in lower_bounds:
            if abs(t-l) <=w2:
                tn = tn-1 #put true neg
                fn = fn+1#remove the old false neg
                
                break
    return tp,tn,fp,fn

def classify_passage(dataframe, ycol="MuStdAmplPaper",gt="Label",plot_roc=True):
    dfPeaks = pd.DataFrame(columns=["Time",ycol])
    for index, row in dataframe.iterrows():
        if index==0 or index == dataframe.tail(1).index:
            continue
        if row[ycol] >= dataframe.iloc[index-1][ycol] and row[ycol] > dataframe.iloc[index+1][ycol]:
            new_row = pd.DataFrame([row[["Time", ycol]]])
            dfPeaks = pd.concat([dfPeaks, new_row], ignore_index=True)
    #function that returns the ROC plot and the AUC (skipping multiple misprediction)
    tau = min(dfPeaks[ycol])
    num_iter = 1000
    step = (max(dfPeaks[ycol]) - tau) / num_iter
    tpr = []
    fpr = []
    while tau < max(dfPeaks[ycol]):
        ttrues = list(dfPeaks.loc[dfPeaks[ycol]>=tau,"Time"])
        tfalses = list(dfPeaks.loc[dfPeaks[ycol]<tau,"Time"])
        tp,tn,fp,fn = checkGT(ttrues,tfalses)
        tpr.append(tp/(tp+fn))
        fpr.append(fp/(fp+tn))
        #print(fp/(fp+tn),tp/(tp+fn))
        
        tau = tau+step
    
    if(plot_roc):
        #Plot the roc curve
        plt.figure(figsize=(3,3),dpi=220)
        plt.plot(fpr, tpr)
        plt.plot([0, 1], [0, 1], color = 'green')
        plt.xlim(-0.05, 1.05)
        plt.ylim(-0.05, 1.05)
        plt.grid()
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC curve")
        plt.show()

    #print(auc(fpr, tpr))
    return auc(fpr, tpr)

def extractWindowedFeatures(data,column_indexes = [],w2=3):
    data["TimeWindow"] = np.floor(data["Timestamp"] / w2)*w2
    #vertical mean/std
    dataStd = data.groupby(by="TimeWindow").std().drop(["Timestamp","Frame_num"],axis=1)
    #dataMean = data.groupby(by="TimeWindow").mean().drop(["Timestamp","Frame_num"],axis=1)
    
    featuredDf = pd.DataFrame()
    featuredDf["Time"] = data["TimeWindow"].unique()
    #horizontal
    featuredDf["MuStdAmplPaper"] = dataStd[[j for j in column_indexes if j.startswith('Ampl')]].mean(axis=1).reset_index(drop=True) #Axis=1: mean over different columns -> into one col
    return featuredDf

def extractWindowedOptimized(dataStd,column_indexes = interestedIndexes,w2=3):
    featuredDf = pd.DataFrame()
    featuredDf["Time"] = dataStd["Time"].unique()
    #horizontal
    featuredDf["MuStdAmplPaper"] = dataStd[[f"Ampl{j}" for j in column_indexes]].mean(axis=1).reset_index(drop=True) #Axis=1: mean over different columns -> into one col
    return featuredDf

#removes outliers from the data
def filterData(df,w1=3,lambda1=3):
    data = df.copy() #clone and return a copy
    #w1 = 5 #best=2
    #lambda1 = 3 #best=4
    #col_list = [f"Ampl{j}" for j in interestedIndexes]
    col_list = [j for j in data.columns if "Ampl" in j]

    for index, row in data.iterrows():
        if index == 0:
            prev_row = row
            continue
        if (index%50 == 0): print(index)
        subDf = data.loc[(data["Timestamp"]<=row['Timestamp']) & (data["Timestamp"]> row['Timestamp'] - w1),col_list]
        means = subDf.mean(axis=0)
        stds = subDf.std(axis=0)

        for c in col_list: 
            if (abs(row[c] - means[c]) / stds[c]) > lambda1:
                data.at[index,c] = prev_row[c]
                #row[c] = prev_row[c]

        prev_row = row
    return data

def complex_real(complex_value):
    return complex(complex_value).real

def complex_imag(complex_value):
    return complex(complex_value).imag

def complex_rebuild(real,imag):
    return (real + 1j*imag)

#Function to get top N features for each principal component
def get_top_n_features(loadings_df, n):
    top_features = {}
    for pc in loadings_df.columns:
        top_features[pc] = loadings_df[pc].abs().sort_values(ascending=False).head(n).index.tolist()
    return top_features

In [ ]:
def lloyd_max_quantization(data, num_levels=16, max_iter=100, delta=1e-6):
    min_val = np.min(data)
    max_val = np.max(data)
    clusters = np.linspace(min_val, max_val, num_levels) #Uniformly spaced 

    for _ in range(max_iter):
        thresholds = (clusters[:-1] + clusters[1:]) / 2 #Defines intervals of clusters
        indices = np.digitize(data, thresholds) #Assign each data point to a cluster
        
        new_clusters = np.array([data[indices == i].mean() for i in range(num_levels)]) #Update clusters to better represent the data
        
        empty_clusters = np.isnan(new_clusters) #Restore previous cluster if empty
        new_clusters[empty_clusters] = clusters[empty_clusters] 

        #stop if changes between iterations are small
        if np.max(np.abs(new_clusters - clusters)) < delta:
            break

        clusters = new_clusters

    #Quantize the data based on the final clusters
    quantized_data = clusters[indices]

    return quantized_data, clusters, thresholds

def dequantize_lloyd_max(quantized_data, clusters, thresholds):
    indices = np.digitize(quantized_data, thresholds, right=True)
    return clusters[indices]

In [ ]:
class Node: 
    def __init__(self, value=None, frequency=0, left=None, right=None):
        self.value = value
        self.frequency = frequency
        self.left = left
        self.right = right

    def __lt__(self, other): #redefined "less than" operator for heapq
        return self.frequency < other.frequency

def build_tree(data):
    heap = [Node(value, frequency) for value, frequency in data.items()]  #Init heap
    heapq.heapify(heap)

    while len(heap) > 1:  #pop two smallest nodes, merge them and push the merged node back
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(frequency=left.frequency + right.frequency, left=left, right=right)
        heapq.heappush(heap, merged) 

    return heap[0] #root

def generate_codes(node, code="", huffman_codes=None):
    if huffman_codes is None: 
        huffman_codes = {}

    if node.value is not None: #leaf node case
        huffman_codes[node.value] = code
        return
    else:
        generate_codes(node.left, code + "0", huffman_codes)
        generate_codes(node.right, code + "1", huffman_codes)
    return huffman_codes

def encode_huffman(data, huffman_codes):
    emptyStr = ""
    return emptyStr.join([huffman_codes[val] for val in data]) 

def decode_huffman(encoded_data, huffman_codes):
    decoded_data = []
    code = ""
    for bit in encoded_data: #traverse the encoded data and searches for the code
        code += bit
        for key, value in huffman_codes.items():
            if value == code: #If found, append the corresponding value to the decoded data, otherwise add another bit to the code
                decoded_data.append(key)
                code = ""
                break
                
    return decoded_data

def apply_huffman_encode_per_feature(data):
    encoded_df = pd.DataFrame()
    huffman_codes = {}

    for col in data.columns:
        freq_per_data = Counter(data[col]) 
        root = build_tree(freq_per_data)
        code = generate_codes(root)
        #print("data["+ col +"]:\n", data[col])
        encoded_df[col] = data[col].apply(lambda x: encode_huffman([x], code))
        huffman_codes[col] = code
    return encoded_df, huffman_codes

def apply_huffman_decode_per_feature(encoded_data, huffman_codes):
    decoded_df = pd.DataFrame()

    for col in encoded_data.columns:
        decoded_df[col] = decode_huffman(''.join(encoded_data[col]), huffman_codes[col])
    return decoded_df

#### PCA Compression

In [ ]:
def data_preprocessing(df, method):
    #df = pd.read_csv('csv/passage.csv')
    df['Timestamp'] = round(df['Timestamp'], 4)
    data = df.copy()
    
    columns_to_drop = (['Frame_num', 'Source_address', 'TimeWindow'] + 
                    [f"Phase{i}" for i in notInterestedIndexes] + 
                    [f"Ampl{i}" for i in notInterestedIndexes] + 
                    [f"CSI{i}" for i in notInterestedIndexes])
    data.drop(columns=columns_to_drop, inplace=True)

    if ignorePhases:
        data.drop(columns=[col for col in data.columns if col.startswith('Phase')], inplace=True); #Removes Phase columns

    if method == Compression_Method.XY:  
        for j in interestedIndexes:
            data[f'X{j}'] = data[f"CSI{j}"].apply(complex_real)
            data[f'Y{j}'] = data[f"CSI{j}"].apply(complex_imag)
        data.drop(columns=[col for col in data.columns if col.startswith(('Ampl', 'Phase'))], inplace=True); #Removes Ampl and Phase columns
    elif method == Compression_Method.AmpPhaseFiltered:
        data = filterData(data)

    data.drop(columns=[col for col in data.columns if col.startswith('CSI')], inplace=True); #Removes CSI columns
    data.set_index('Timestamp', inplace=True)
    print("Number of features:", len(data.columns))
    
    return data

Check how many components are needed to have an explanation of 95% of the variance

In [ ]:
def find_n_components(data, target, directory):
    #Fit and transform the data
    scaled_data = scaler.fit_transform(data)

    #Apply PCA
    pca = PCA()
    pca.fit(scaled_data)

    var_cumulative = np.cumsum(pca.explained_variance_ratio_)*100

    #finds PCs that explain 95% of the variance
    k = np.argmax(var_cumulative > target) + 1
    print(f"Number of components explaining {target}% variance: "+ str(k))

    plt.figure(figsize=(10, 5))
    plt.title('Cumulative Explained Variance explained by the components')
    plt.ylabel('Cumulative Explained variance')
    plt.xlabel('Principal components')
    plt.axvline(x=k, color="r", linestyle="--")
    plt.axhline(y=target, color="r", linestyle="--")
    plt.plot(range(1, pca.n_components_ + 1), var_cumulative, marker='o', linestyle='--')
    plt.grid()
    plt.savefig(os.path.join(directory, 'var_cumulative_x_component.png'))
    plt.show()

    return scaled_data, k

Apply PCA, check the explained variance ratio and the cumulative explained variance ratio

In [ ]:
def analyze_PCA(scaled_data, n_components, directory):
    pca = PCA(n_components=n_components)
    reduced_data = pca.fit_transform(scaled_data)

    reduced_df = pd.DataFrame(data=reduced_data, columns=[f'PC{i}' for i in range(n_components)])

    #Explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_
    print("Explained variance ratio:", explained_variance_ratio)

    #Cumulative explained variance
    cumulative_explained_variance = np.cumsum(explained_variance_ratio)
    print("Final Cumulative Explained Variance:", cumulative_explained_variance[-1])

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, n_components + 1), cumulative_explained_variance, marker='o', linestyle='--')
    plt.title('Cumulative Explained Variance by PCA Components')
    plt.xlabel('Number of Principal Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.grid()
    plt.savefig(os.path.join(directory, 'zoomed_var_cumulative_x_component.png'))
    plt.show()
    
    return reduced_df, pca

For each Principal Component, find the top "n" features that contribute most to the variance of that component.

In [ ]:
def analyze_PC(data, pca, n_components):
    loadings = pca.components_
    loadings_df = pd.DataFrame(data=loadings.T, index=data.columns, columns=[f'PC{i+1}' for i in range(loadings.shape[0])])
    column = []

    top_n_features = get_top_n_features(loadings_df, n_components)

    for pc, features in top_n_features.items():
        #print(f"Top {n_components} features for {pc}: {features}") #uncomment to see the top features per PC
        for feature in features:
            if feature not in column:
                column.append(feature)
    print("available features: ", len(data.columns))
    print("features used: ", len(column))

    difference = set(data.columns) - set(column)
    print("Unused Features:", difference)

    return difference

#### Lloyd-Max Quantization 

In [ ]:
def apply_quantization(reduced_df, lvls):
    df_quantized = reduced_df.apply(lambda col: lloyd_max_quantization(col.values, num_levels=lvls)[0])
    return df_quantized


#YET TO TEST
"""
clusters, thresholds = lloyd_max_quantization(reduced_df.values)[1:3]

print("Centroids:", clusters)
print("Thresholds:", thresholds)

#Dequantize the quantized data
df_dequantized = df_quantized.apply(lambda col: dequantize_lloyd_max(col.values, clusters, thresholds))

#Reconstruct the original data
reconstructed_data = pca.inverse_transform(df_dequantized.values)
df_reconstructed = scaler.inverse_transform(reconstructed_data)

df_reconstructed = pd.DataFrame(data=df_reconstructed, columns=data.columns)

print("Original data:")
data.reset_index(inplace=True)
data.drop(columns=['Timestamp'], inplace=True)
print(data)
print("Reconstruction error:", np.mean((data.values - df_reconstructed.values)**2))
print(df_reconstructed)
"""

#### Entropy Coding (Huffman)

In [ ]:
def apply_encoding(df_quantized, df):
    encoded_df, huffman_codes = apply_huffman_encode_per_feature(df_quantized)
    encoded_df = pd.concat([df[['Frame_num', 'Timestamp']], encoded_df, df['TimeWindow']], axis=1)

    return encoded_df, huffman_codes

In [ ]:
def apply_decoding(encoded_df, huffman_codes):
    decoded_df = apply_huffman_decode_per_feature(encoded_df.iloc[:, 2:-1], huffman_codes)
    return decoded_df

#### Reconstruction

Reconstruct the dataset (without CSI components) and save it in csv

In [ ]:
def reconstruct_data(decoded_df, encoded_df, pca, scaler, data):

    df_scaled_reconstructed = pca.inverse_transform(decoded_df.values)
    df_reconstructed = scaler.inverse_transform(df_scaled_reconstructed)

    """
    print('Original data shape:', df.shape)
    print('Scaled data shape:', reduced_data.shape)
    print('PCA components shape:', reduced_df.shape)
    print('Reconstructed scaled data shape:', df_scaled_reconstructed.shape)
    print('Reconstructed original data shape:', df_reconstructed.shape)
    """
    
    df_reconstructed = pd.DataFrame(df_reconstructed, columns=data.columns)
    df_reconstructed = pd.concat([encoded_df.iloc[:, [0, 1, -1]], df_reconstructed], axis=1)

    if method == Compression_Method.XY:
        for j in interestedIndexes:
            df_reconstructed[f'CSI{j}'] = df_reconstructed.apply(lambda x: complex_rebuild(x[f'X{j}'], x[f'Y{j}']), axis=1)
                
            #compute back ampl and phases
            df_reconstructed[f'Ampl{j}'] = df_reconstructed[f'CSI{j}'].apply(abs)
            df_reconstructed[f'Phase{j}'] = df_reconstructed[f'CSI{j}'].apply(cmath.phase)

        df_reconstructed.drop(columns=[f'X{j}' for j in interestedIndexes], inplace=True)
        df_reconstructed.drop(columns=[f'Y{j}' for j in interestedIndexes], inplace=True)
        
    return df_reconstructed


In [ ]:
def plot_ampl_comparison(data, reconstruct_data, column, directory):

    data = data[(data['Timestamp'] >= 0) & (data['Timestamp'] <= 700)]
    reconstruct_data = reconstruct_data[(reconstruct_data['Timestamp'] >= 0) & (data['Timestamp'] <= 700)]
    
    plt.figure(figsize=(20, 6))

    columns = [f'Ampl{column}', f'Ampl-{column}'] 

    for i in range(columns.__len__()):
        if (columns[i] not in data.columns) or (columns[i] not in reconstruct_data.columns):
            print(f"Column {columns[i]} not found in the data")
            return

        # Plot the original data
        plt.plot(data['Timestamp'], data[columns[i]], label=f'Original {columns[i]}', color="red" if i == 0 else "green")

        # Plot the reconstructed data
        plt.plot(reconstruct_data['Timestamp'], reconstruct_data[columns[i]], label=f'Reconstructed {columns[i]}', color="blue" if i == 0 else "orange")
    
    # Add plot details
    plt.title('Amplitude Comparison')
    plt.xlabel('Timestamp')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(directory, f'Ampl{column}_Ampl-{column}_comparison.png'))
    plt.show()


#### Classification (partially from another notebook):

In [ ]:
def load_comparison():
    filteredFeaturesPassage = pd.read_csv("csv/filteredFeaturesPassage.csv")
    #featuredDf = extractWindowedFeatures(filteredDf,column_indexes = interestedIndexes,w2=w2)
    orig_auc = classify_passage(filteredFeaturesPassage, plot_roc=False)
    return orig_auc

In [ ]:
def apply_filtering(df_reconstructed):
    reconstructedPassage = df_reconstructed

    if method == Compression_Method.AmpPhaseFiltered:
        reconstructed_filtered = reconstructedPassage
    else:
        reconstructed_filtered = filterData(reconstructedPassage) #removes outliers
    reconstructed_filtered.drop(columns=[col for col in reconstructed_filtered.columns if col.startswith('CSI')], inplace=True); #Removes CSI columns

    return reconstructed_filtered

In [ ]:
def apply_classification(reconstructed_filtered):
    #compute features
    reconstructed_featured = extractWindowedFeatures(reconstructed_filtered,column_indexes = reconstructed_filtered.columns,w2=w2)
    
    #classify
    print(classify_passage(reconstructed_featured,plot_roc=False))

    return reconstructed_featured, classify_passage(reconstructed_featured,plot_roc=False)

#### Results
8 levels:
- **AmpPhaseFiltered**: 0.9705615942028984 (PCA 0.95, LLoyd-Max: num_levels=**8**, max_iter=100, tol=1e-6) 
- **AmpPhase**: 0.9716093445814408 (PCA 0.95, LLoyd-Max: num_levels=**8**, max_iter=100, tol=1e-6)
- **XY**: 0.9370967741935484 (PCA 0.95, LLoyd-Max: num_levels=**8**, max_iter=100, tol=1e-6)
---
16 levels:
- **AmpPhaseFiltered**: 0.9661764705882354 (PCA 0.95, LLoyd-Max: num_levels=**16**, max_iter=100, tol=1e-6) 
- **AmpPhase**:  (PCA 0.95, LLoyd-Max: num_levels=**16**, max_iter=100, tol=1e-6)
- **XY**:  (PCA 0.95, LLoyd-Max: num_levels=**16**, max_iter=100, tol=1e-6)
---
Only Ampl:
- **AmpPhaseFiltered**: (PCA 0.95, LLoyd-Max: num_levels=8 max_iter=100, tol=1e-6) 
- **AmpPhase**: 0.971077504725898 (PCA 0.95, LLoyd-Max: num_levels=8 max_iter=100, tol=1e-6) 
- **XY**:  (PCA 0.95, LLoyd-Max: num_levels=8 max_iter=100, tol=1e-6)

#### Useless

Assumptions:
- Cleaned dataset to work only on needed columns then restores it before saving it
- 95% explained variance treshold
- 8 e 16 quantization lvls
- 10^-6 delta of values in Lloyd-Max

Questions:
- Is it okay to apply PCA on the whole dataset? it's not big
- Is it okay to use the same windows that the paper used or should I try to find the optimal ones (like the paper did?)
- Is it better to have a single huffman tree for the whole dataset or one per feature? (I think the second one is better)
- Is it okay to use 8/16 quantization lvl or should I try to find the best one first and then use only that?
- How to store the data (i guess it's better to store the encoded data + the huffman code, PCA matrix, scaler matrix and eventually the quantization matrix)

TODO:
- Improve reconstruction to reduce the amount of data needed
- Test 16 lvl
- Test diff tol
- Find best step_num
- all tests with dequantized dataset (in theory it approximates og data more closely than the quantized version)

Optimizations:
- Is FrameNum necessary if we already have timestamp?
- Timestamps decimals can be shortened

In [ ]:
df = pd.read_csv('csv/passage.csv')
data = data_preprocessing(df, method)

In [ ]:
directory = 'csv/step_' + str(method.value)
os.makedirs(directory, exist_ok=True)

df = pd.read_csv('csv/passage.csv')
data = data_preprocessing(df, method)

target = 95
num_levels = 8

#PCA
scaled_data, n_components = find_n_components(data, target)
reduced_df, pca = analyze_PCA(scaled_data, n_components, target)
unused_features = analyze_PC(data, pca, n_components)

#LLoyd-Max Quantization
quantized_df = apply_quantization(reduced_df, num_levels)

#Encode-Decode
encoded_df, huffman_codes = apply_encoding(quantized_df, df)
decoded_df = apply_decoding(encoded_df, huffman_codes)
print("Original DataFrame equals Decoded DataFrame:", quantized_df.equals(decoded_df)) #Correctness check

#Reconstruction
reconstructed_df = reconstruct_data(decoded_df, encoded_df, pca, scaler, data)
load_comparison()

columns = [i for i in interestedIndexes if i >= 0]
for column in columns:
    plot_ampl_comparison(df, reconstructed_df, column, num_levels, target)

reconstructed_df.drop(columns=unused_features, inplace=True)


In [ ]:
#Filtering and Classification
reconstructed_filtered = apply_filtering(reconstructed_df)
reconstructed_featured, auc = apply_classification(reconstructed_filtered)

#Save CSV
if(saveCSV):
    encoded_df.to_csv(os.path.join(directory, 'encodedQuantizedPCAPassage.csv'), index=False)
    #reconstructed_df.set_index('Timestamp', inplace=True)
    reconstructed_df.to_csv(os.path.join(directory, 'passage_reconstructed.csv'))
    reconstructed_filtered.to_csv(os.path.join(directory, 'filteredPassage_reconstructed.csv'), index=False)
    reconstructed_featured.to_csv(os.path.join(directory, 'filteredFeaturesPassage_reconstructed.csv'), index=False)

In [ ]:
# Define your targets and num_levels lists
targets = [80, 85, 90, 95, 99]
num_levels = [4, 8, 16, 32]

results = []
orig_auc = load_comparison()

# Iterate over targets and num_levels
for target in targets:
    directory = os.path.join(base_directory, f'Target_{target}')
    os.makedirs(directory, exist_ok=True)
    
    # Read and preprocess data
    df = pd.read_csv('csv/passage.csv')
    data = data_preprocessing(df, method)
    
    # PCA
    scaled_data, n_components = find_n_components(data, target, directory)
    reduced_df, pca = analyze_PCA(scaled_data, n_components, directory)
    unused_features = analyze_PC(data, pca, n_components)

    for level in num_levels:
         # Create a directory for each combination of target and num_levels
        sub_directory = os.path.join(directory, f'lvls_{level}')
        os.makedirs(sub_directory, exist_ok=True)

        # Lloyd-Max Quantization
        quantized_df = apply_quantization(reduced_df, level)

        # Encode-Decode
        encoded_df, huffman_codes = apply_encoding(quantized_df, df)
        decoded_df = apply_decoding(encoded_df, huffman_codes)
        print("Original DataFrame equals Decoded DataFrame:", quantized_df.equals(decoded_df)) #Correctness check

        # Reconstruction
        reconstructed_df = reconstruct_data(decoded_df, encoded_df, pca, scaler, data)
        print("original_auc:",orig_auc)

        columns = [i for i in interestedIndexes if i >= 0]
        for column in columns:
            plot_ampl_comparison(df, reconstructed_df, column, sub_directory)

        reconstructed_df.drop(columns=unused_features, inplace=True)

        # Filtering and Classification
        reconstructed_filtered = apply_filtering(reconstructed_df)
        print(f"----------Results with target {target}%, num_levels {level} ----------")
        reconstructed_featured, auc_value = apply_classification(reconstructed_filtered)

        results.append({
            'Method': method.name,
            'num_levels': level,
            'target': target,
            'AUC': auc_value
        })

        # Save CSVs in the specific directory
        if saveCSV:
            encoded_df.to_csv(os.path.join(sub_directory, 'encodedQuantizedPCAPassage.csv'), index=False)
            #reconstructed_df.set_index('Timestamp', inplace=True)
            reconstructed_df.to_csv(os.path.join(sub_directory, 'passage_reconstructed.csv'))
            reconstructed_filtered.to_csv(os.path.join(sub_directory, 'filteredPassage_reconstructed.csv'), index=False)
            reconstructed_featured.to_csv(os.path.join(sub_directory, 'filteredFeaturesPassage_reconstructed.csv'), index=False)

# Convert results list to DataFrame
results_df = pd.DataFrame(results)

# Save the results DataFrame to a CSV file
results_file = os.path.join(base_directory, 'classification_results.csv')
results_df.to_csv(results_file, index=False)

